Loading libraries

In [ ]:
pip install wget

In [ ]:
pip install py7zr

In [ ]:
import xml.etree.ElementTree as et
import pandas as pd
import wget

In [ ]:
import py7zr

Download Stack Overflow Posts from https://ia800107.us.archive.org/view_archive.php?archive=/27/items/stackexchange/stackoverflow.com-Posts.7z

In [ ]:
wget.download("https://archive.org/download/stackexchange/stackoverflow.com-Posts.7z")

Unzipping posts

In [ ]:
py7zr x stackoverflow.com-Posts.7z

In [ ]:
postPath = "Posts.xml"

In [ ]:
androidSamples = []
awsSamples = []
azureSamples = []
springSamples = []

For each post in Posts, we try to find the github referrence in Post body.

In [ ]:
with open(postPath, encoding='utf-8') as posts:
    for line in posts:
        if "<?xml" in line or "<posts>" in line:
            continue
        parsed = et.fromstring(line)
        if parsed.get("PostTypeId") != "1":
            continue
        id = int(parsed.get("Id"))
        if id % 100000 == 0:
            print(id)
        if "github.com/googlesamples" in parsed.get("Body"):
            androidSamples.append(parsed)
        if "github.com/aws-samples" in parsed.get("Body"):
            awsSamples.append(parsed)
        if "github.com/Azure-Samples" in parsed.get("Body"):
            azureSamples.append(parsed)
        if "github.com/spring-guides" in parsed.get("Body"):
            springSamples.append(parsed)

Path of code samples by framework

In [ ]:
androidSamplesPath = "listOfSamples/googlesamples.txt"
awsSamplesPath = "listOfSamples/awssamples.txt"
azureSamplesPath = "listOfSamples/azuresamples.txt"
springSamplesPath = "listOfSamples/springsamples.txt"

Filtring only questions about the selected code samples

In [ ]:
def filterSamples(framework, samplesPath, questions):
    lines = []
    with open(samplesPath) as file:
        for line in file:
            line = line.replace("\n", "")
            for question in questions:
                if line in question.get("Body"):
                    print(f"{line} === {question.get('Body')}")
                    lines.append([framework, line, question.get("Id"), question.get("CreationDate"), question.get("OwnerUserId"), question.get("PostTypeId"), question.get("AcceptedAnswerId")])
    return lines

In [ ]:
def createDataFrame(lines):
    columns = ["framework", "path", "id", "creationDate", "ownerUserId", "postTypeId", "acceptedAnswerId"]
    df = pd.DataFrame(lines, columns=columns)
    return df

Transforming data to DataFrame

In [ ]:
androidDF = createDataFrame(filterSamples("android", androidSamplesPath, androidSamples))

In [ ]:
awsDF = createDataFrame(filterSamples("aws", awsSamplesPath, awsSamples))

In [ ]:
azureDF = createDataFrame(filterSamples("azure", azureSamplesPath, azureSamples))

In [ ]:
springDF = createDataFrame(filterSamples("spring", springSamplesPath, springSamples))

Saving questions as csv

In [ ]:
androidDF.to_csv("questions/androidQuestions.csv", index=False)

In [ ]:
awsDF.to_csv("questions/awsQuestions.csv", index=False)

In [ ]:
azureDF.to_csv("questions/azureQuestions.csv", index=False)

In [ ]:
springDF.to_csv("questions/springQuestions.csv", index=False)

Saving code samples with have questions

In [ ]:
androidDF["path"].drop_duplicates().to_csv("samplesWithQuestions/androidSamples.csv", index=False)

In [ ]:
awsDF["path"].drop_duplicates().to_csv("samplesWithQuestions/awsSamples.csv", index=False)

In [ ]:
azureDF["path"].drop_duplicates().to_csv("samplesWithQuestions/azureSamples.csv", index=False)

In [ ]:
springDF["path"].drop_duplicates().to_csv("samplesWithQuestions/springSamples.csv", index=False)